In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import json
import time

# Path to your ChromeDriver executable
chromedriver_path = "Your/chrome/driver/path

# User credentials
username = 'your/username'
password = 'your/paasword'
# Twitter user ID for profile extraction will be provided as input later

# Set up Chrome options
options = webdriver.ChromeOptions()
options.add_argument('--headless')     # Run browser in headless mode (without UI)
options.add_argument('--disable-gpu')  # Disable GPU acceleration

# Initialize WebDriver service
service = Service(executable_path=chromedriver_path)

# Launch browser
driver = webdriver.Chrome(service=service, options=options)

def login_twitter(username, password):
    """
    Log in to Twitter using provided username and password.
    """
    driver.get('https://twitter.com/login')
    time.sleep(3)
    
    # Enter username/email
    username_input = driver.find_element(By.NAME, 'text')
    username_input.send_keys(username)
    username_input.send_keys(Keys.RETURN)
    time.sleep(2)
    
    # Enter password
    password_input = driver.find_element(By.NAME, 'password')
    password_input.send_keys(password)
    password_input.send_keys(Keys.RETURN)
    time.sleep(5)

# Perform login
login_twitter(username, password)

def extract_twitter_info(user_id, max_tweets=100):
    """
    Extract tweets, user name, and bio from a Twitter profile.
    
    :param user_id: Twitter user ID (username)
    :param max_tweets: Maximum number of tweets to collect
    :return: Dictionary containing user name, bio, and a list of tweets
    """
    # Construct the profile URL
    profile_url = f'https://twitter.com/{user_id}'
    
    # Navigate to the user's profile page
    driver.get(profile_url)
    time.sleep(5)
    
    collected_tweets = []  # List to store tweets
    last_height = driver.execute_script("return document.body.scrollHeight")

    while len(collected_tweets) < max_tweets:
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'html.parser')
        
        # Extract tweet elements using data-testid attribute
        tweet_elements = soup.find_all('div', {'data-testid': 'tweetText'})
        for tweet in tweet_elements:
            tweet_text = tweet.get_text(strip=True)
            if tweet_text not in collected_tweets:
                collected_tweets.append(tweet_text)
                if len(collected_tweets) >= max_tweets:
                    break
        
        # Scroll down to load more tweets
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(5)
        
        # Check if scrolling has reached the bottom
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            print("No more tweets to load.")
            break
        last_height = new_height
    
    # Extract user name and bio
    user_name_element = soup.find('div', {'data-testid': 'UserName'})
    if user_name_element:
        user_name = user_name_element.find('div').find('span').text
    else:
        user_name = "N/A"
    
    bio_element = soup.find('div', {'data-testid': 'UserDescription'})
    bio = bio_element.text if bio_element else "N/A"

    # Format the collected data
    profile_data = {
        "user_name": user_name,
        "bio": bio,
        "tweets": collected_tweets[:max_tweets]
    }
    
    return profile_data

# Get Twitter user ID input from the user
user_id = input("Enter the Twitter user ID: ")

# Extract profile data
profile_data = extract_twitter_info(user_id, max_tweets=100)

# Save extracted data to a JSON file
with open(f'{user_id}.json', 'w', encoding='utf-8') as f:
    json.dump(profile_data, f, ensure_ascii=False, indent=4)
